# Frame The Problem

https://www.cs.toronto.edu/~kriz/cifar.html

# Frame The Problem
1. **Define the objective in business terms.**
- We have been hired by the National Wildlife Service, they have cameras they want to use to track staple wild animal populations, specifically birds, deer, and frogs, with their cameras and statistical models. Cats and dogs can also affect the environment, so they make sure people's pets do not kill off the wild animal populations. Other images of horses, airplanes, automobiles, ships, and trucks are often caught on these cameras and are of little use to the service, but they want to be tracked because some papers have suggested that noise pollution from these can affect animal and plant populations.
2. **How will your solution be used?**
- Our solution will predict what pictures are in bulk allowing National Wildlife Service to plug the numbers into their statistically models.
3. **What are the current solutions/workarounds (if any)?**
- Currently they have unpaid interns look at photos one by one labeling them to count animals.
4. **How should you frame this problem (supervised/unsupervised, online/offline, ...)?**
- It is a supervised task, it does not need to be online.
5. **How should performance be measured? Is the performance measure aligned with the business objective?**
6. **What would be the minimum performance needed to reach the business objective?**
7. **What are comparable problems? Can you reuse experience or tools?**
8. **Is human expertise available?**
9. **How would you solve the problem manually?**
10. **List the assumptions you (or others) have made so far. Verify assumptions if possible.**

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from tensorflow import keras
import tensorflow as tf
from functools import partial



# Get the Data

In [35]:
label_words = {0:"airplane", 1:"automobile", 2:"bird", 3:"cat", 4:"deer", 5:"dog", 6:"frog", 7:"horse", 8:"ship", 9:"truck"}
def get_label(index:list):
    labels = []
    for i in index:
        labels.append(label_words[i])
    return labels

def unpickle(has_validation=True):
    files = ['data_batch_1', 'data_batch_2', 'data_batch_3','data_batch_4']
    if has_validation:
        X_train = []
        y_train = []
        X_val = []
        y_val = []
        data = None
        for file in files:
            with open('data/'+file, 'rb') as fo:
                data = pickle.load(fo, encoding='bytes')
                y_train += list(data[b'labels'])
                X_train += list(data[b'data'])

        with open('data/data_batch_4', 'rb') as fo:
            data = pickle.load(fo, encoding='bytes')
            y_val = list(data[b'labels'])
            X_val = list(data[b'data'])

        with open('data/test_batch', 'rb') as fo:
            data = pickle.load(fo, encoding='bytes')
            y_test = list(data[b'labels'])
            X_test = list(data[b'data'])

        return X_train, y_train, X_val, y_val, X_test, y_test
    else:
        files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
        y_train = []
        X_train = [] 
        data = None
        for file in files:
            with open('data/'+file, 'rb') as fo:
                data = pickle.load(fo, encoding='bytes')
                y_train += list(data[b'labels'])
                X_train += list(data[b'data'])

        with open('data/test_batch', 'rb') as fo:
            data = pickle.load(fo, encoding='bytes')
            y_test = list(data[b'labels'])
            X_test = list(data[b'data'])

        return X_train, y_train, X_test, y_test
    

def plot_cifar10_images(images, labels):
    axes = plt.subplots(5, 5, figsize=(10, 10))[1]
    for i in range(5):
        for j in range(5):
            index = i * 5 + j
            image = images[index].reshape(3, 32, 32).transpose(1, 2, 0)
            axes[i, j].imshow(image)
            axes[i, j].set_title(labels[index])
            axes[i, j].axis('off')
    plt.show()

In [36]:
X_train, y_train, X_valid, y_valid, X_test, y_test = unpickle(has_validation=True)
# Reshape the data
X_train = np.array(X_train).reshape(-1, 32, 32, 3)
X_valid = np.array(X_valid).reshape(-1, 32, 32, 3)
X_test = np.array(X_test).reshape(-1, 32, 32, 3)
# Normalize the data
X_train = X_train / 255.0
X_valid = X_valid / 255.0
X_test = X_test / 255.0
# Reshape the labels
y_train = np.array(y_train).reshape(-1, 1)
y_valid = np.array(y_valid).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

# Explore the Data

# Short Listing

In [ ]:
# basic MLP
model = keras.models.Sequential([
    keras.layers.Input(shape=[32, 32, 3]),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dense(50, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    batch_size=32
)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2342 - loss: 2.0737 - val_accuracy: 0.3408 - val_loss: 1.8356
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 882us/step - accuracy: 0.3531 - loss: 1.8138 - val_accuracy: 0.3852 - val_loss: 1.7387
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 879us/step - accuracy: 0.3878 - loss: 1.7164 - val_accuracy: 0.4082 - val_loss: 1.6678
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 885us/step - accuracy: 0.4071 - loss: 1.6680 - val_accuracy: 0.4290 - val_loss: 1.6234
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 878us/step - accuracy: 0.4197 - loss: 1.6182 - val_accuracy: 0.4461 - val_loss: 1.5826
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 881us/step - accuracy: 0.4352 - loss: 1.5891 - val_accuracy: 0.4576 - val_loss: 1.5448
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 924us/step - accuracy: 0.4457 - loss: 1.5653 - val_accuracy: 0.4584 - val_loss: 1.5578
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 879us/step - accuracy: 0.4491 - l

In [38]:
# selu activation "lecun_normal"
model = keras.models.Sequential([
    keras.layers.Input(shape=[32, 32, 3]),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.Dense(50, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.Dense(50, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.Dense(50, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.Dense(50, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    X_train, y_train,
    epochs=20,
    validation_data=(X_valid, y_valid),
    batch_size=32,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3)
    ]
)

Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2382 - loss: 2.0852 - val_accuracy: 0.3536 - val_loss: 1.7982 - learning_rate: 0.0100
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3602 - loss: 1.7735 - val_accuracy: 0.3790 - val_loss: 1.7184 - learning_rate: 0.0100
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3937 - loss: 1.6791 - val_accuracy: 0.4067 - val_loss: 1.6691 - learning_rate: 0.0100
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4228 - loss: 1.6139 - val_accuracy: 0.4393 - val_loss: 1.5708 - learning_rate: 0.0100
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4338 - loss: 1.5800 - val_accuracy: 0.4401 - val_loss: 1.5702 - learning_rate: 0.0100
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4463 - loss: 1.5391 - val_accuracy: 0.4590 - val_loss: 1.5182 - learning_rate: 0.0100
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4572 - l

In [46]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal')
# convolutional neural network
model = keras.models.Sequential([
    keras.layers.Input(shape=[32, 32, 3], name='input'),
    DefaultConv2D(filters=32, name='conv1'),
    keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool1'),
    DefaultConv2D(filters=64, name='conv2'),
    keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool2'),
    keras.layers.Flatten(name='flatten'),
    keras.layers.Dense(128, activation='relu', name='dense1'),
    keras.layers.Dense(10, activation='softmax', name='output')
])

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']

)
history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    batch_size=32, 
    callbacks=[
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.2661 - loss: 2.0215 - val_accuracy: 0.4306 - val_loss: 1.6377
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.4487 - loss: 1.5744 - val_accuracy: 0.4826 - val_loss: 1.4799
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.5000 - loss: 1.4216 - val_accuracy: 0.5146 - val_loss: 1.3924
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.5427 - loss: 1.3134 - val_accuracy: 0.5519 - val_loss: 1.2863
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.5626 - loss: 1.2457 - val_accuracy: 0.5766 - val_loss: 1.1996
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.5918 - loss: 1.1796 - val_accuracy: 0.6233 - val_loss: 1.1082
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.6083 - loss: 1.1265 - val_accuracy: 0.6246 - val_loss: 1.0835
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.6315 -

In [45]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal')
# convolution with dropout
model = keras.models.Sequential([
    keras.layers.Input(shape=[32, 32, 3], name='input'),
    DefaultConv2D(filters=64, name='conv1'),
    keras.layers.MaxPooling2D(name='pool1'),
    DefaultConv2D(filters=128, name='conv2'),
    DefaultConv2D(filters=128, name='conv3'),
    keras.layers.MaxPooling2D(name='pool2'),
    keras.layers.Flatten(name='flatten'),
    keras.layers.Dense(128, activation='relu', name='dense1', kernel_initializer='he_normal'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu', name='dense2', kernel_initializer='he_normal'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax', name='output')
])

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],


)
history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    batch_size=32, 
    callbacks=[
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 53s 42ms/step - accuracy: 0.1457 - loss: 2.2719 - val_accuracy: 0.3377 - val_loss: 1.9098
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 53s 42ms/step - accuracy: 0.2596 - loss: 1.9978 - val_accuracy: 0.3878 - val_loss: 1.7471
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 52s 41ms/step - accuracy: 0.3212 - loss: 1.8554 - val_accuracy: 0.4383 - val_loss: 1.5949
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 51s 41ms/step - accuracy: 0.3761 - loss: 1.7387 - val_accuracy: 0.4596 - val_loss: 1.5330
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 51s 41ms/step - accuracy: 0.4015 - loss: 1.6651 - val_accuracy: 0.4949 - val_loss: 1.4298
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 51s 41ms/step - accuracy: 0.4381 - loss: 1.5824 - val_accuracy: 0.5470 - val_loss: 1.3317
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 51s 41ms/step - accuracy: 0.4625 - loss: 1.5121 - val_accuracy: 0.5531 - val_loss: 1.2887
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 53s 43ms/step - accuracy: 0.4833 -